In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split

In [108]:
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/dataset1.xlsx")
data.shape

(1994, 121)

In [109]:
Normdata = data
Normdata = np.matrix(Normdata)
cols = Normdata.shape[1]

X = Normdata[:,1:cols-1]
y = Normdata[:,cols-1:cols]

In [110]:
X = np.matrix(X)
y = np.matrix(y)

(1994L, 119L)

In [112]:
for i in range(len(y)):
    if y[i] == 0:
        y[i] = -1

In [114]:
def initialiseLMC():
    c = np.zeros(100000)
    V = np.zeros((100000, X_train.shape[1]-1))
    V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    k = 0
    return V, c, k

In [115]:
def trainLMC(X_train, y_train, V, c, t, m, k):
    #print 'Training Started'
    m = X_train.shape[0]
    n = X_train.shape[1]
    c = np.zeros(100000)
    #V = np.zeros((996, n))
    #V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    for e in range(t):
        #print e
        for i in range(0, m-1):
            y_pred = np.sign(np.dot(X_train[i], V[k].T))
            if y_pred == y_train[i]:
                c[k+1] = c[k] + 1
            else:
                d = np.asscalar(y_train[i])
                p = V[k] + (d*X_train[i]).T
                V[k+1] = V[k] + (d*X_train[i])
                c[k+1] = 1
                k = k + 1
    #print 'Done training' 
    #print k
    return V, c, k

In [116]:
def predictLMC(X_test, V, c, k):
    y_LMC = np.zeros(X_test.shape[0])
    a = np.zeros(k)
    for j in range(0, X_test.shape[0]):
        for i in range(0, k):
            a[i] = c[i] * np.sign(np.dot(X_test[j], V[i].T))
            s = np.sum(a)
            y_LMC[j] = np.sign(s)
    #print 'Test Data Created'
    return y_LMC

In [132]:
epoch = [1, 2, 3, 4, 5, 10, 15, 20, 25, 50, 75, 100]
count = 0
prec = np.zeros((12,2))
recal = np.zeros((12,2))
f1 = np.zeros((12,2))
acc = np.zeros((12,2))
auc =  np.zeros((12,2))
K = np.zeros((12,2))
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn import metrics
for i in range(8,12):
    t = epoch[i]
    print t
    for l in range(0,2):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
        X_train = np.insert(X_train, 0, values=np.ones(X_train.shape[0]), axis=1)
        X_test = np.insert(X_test, 0, values=np.ones(X_test.shape[0]), axis=1)
        V, c, k= initialiseLMC()
        V, c, k = trainLMC(X_train, y_train, V, c, t, X_train.shape[0], k)
        K[count, l] = k
        y_LMC = predictLMC(X_test, V, c, k)
        y_LMC[y_LMC == 0] = 1
        y_pred = y_LMC
        prec[count, l] = precision_score(y_test, y_pred, average="weighted")
        recal[count, l] = recall_score(y_test, y_pred, average="weighted")
        f1[count, l] = f1_score(y_test, y_pred, average="weighted")
        acc[count, l] = accuracy_score(y_test, y_pred)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred) 
        auc[count, l] = metrics.auc(fpr, tpr)
        #print t, count , l, acc[count, l], k
    print t, (K[count, 0]+K[count,1])*1.0/2, (prec[count, 0]+prec[count,1])*1.0/2, (recal[count, 0]+recal[count,1])*1.0/2, (f1[count, 0]+f1[count,1])*1.0/2, (acc[count, 0]+acc[count,1])*1.0/2,(auc[count, 0]+auc[count,1])*1.0/2
    count += 1

25
25 6816.0 0.828676311688 0.828986960883 0.828800404037 0.828986960883 0.821264298835
50
50 12862.0 0.820983916825 0.820962888666 0.820383782899 0.820962888666 0.81280738967
75
75 17842.5 0.84284137521 0.843029087262 0.841810914118 0.843029087262 0.831625637828
100
100 24120.0 0.850759759353 0.851053159478 0.850829765266 0.851053159478 0.844719564202
